In [1]:
from keras_vggface.vggface import VGGFace
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.utils import preprocess_input
from keras.models import Model
from keras.models import load_model
from keras import Sequential
from keras import Input
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,GlobalMaxPooling2D,MaxPooling2D,Dropout
from keras import optimizers
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras import callbacks

import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def focal_loss(labels, logits, gamma=2.0, alpha=4.0):
    """
    focal loss for multi-classification
    FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
    Notice: logits is probability after softmax
    gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
    d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
    :param labels: ground truth labels, shape of [batch_size]
    :param logits: model's output, shape of [batch_size, num_cls]
    :param gamma:
    :param alpha:
    :return: shape of [batch_size]
    """
    epsilon = 1.e-9
    labels = tf.to_int64(labels)
    labels = tf.convert_to_tensor(labels, tf.int64)
    logits = tf.convert_to_tensor(logits, tf.float32)
    num_cls = logits.shape[1]

    model_out = tf.add(logits, epsilon)
    onehot_labels = tf.one_hot(labels, num_cls)
    ce = tf.multiply(onehot_labels, -tf.log(model_out))
    weight = tf.multiply(onehot_labels, tf.pow(tf.subtract(1., model_out), gamma))
    fl = tf.multiply(alpha, tf.multiply(weight, ce))
    reduced_fl = tf.reduce_max(fl, axis=1)
    # reduced_fl = tf.reduce_sum(fl, axis=1)  # same as reduce_max
    return reduced_fl

In [3]:
base_model = VGGFace(model = 'vgg16', weights = 'vggface',input_shape=None,include_top=False)
#base_model=InceptionV3(weights='imagenet', include_top=False)
x=base_model.output
#x_input = Input(shape=(224,224,3))
#x =  (conv_base)(x_input)
x = GlobalAveragePooling2D()(x)
#x = Dense(256,activation= 'relu')(x)
#x=Dropout(rate=0.5)(x)
predictions = Dense(5,activation='softmax')(x)
model = Model(inputs=base_model.input,outputs=predictions)

#opt = optimizers.Adam(lr=0.0001)
#model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [4]:
checkpointer=callbacks.ModelCheckpoint(filepath=
                                       "models/vggface_classifier/fold4_weights.{epoch:02d}-{val_loss:.2f}.hdf5",
                                       monitor='val_loss',verbose=1)

In [4]:
#tbCallBack = callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

In [5]:
train_datagen = ImageDataGenerator(samplewise_center=True,samplewise_std_normalization=True,
                                   rescale=1./255,horizontal_flip=True)
"""
                                   rotation_range=45,
                                   width_shift_range=0.2,height_shift_range=0.2, 
                                   
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   ,vertical_flip=True,
                                  fill_mode="nearest")

"""

                                   
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/home/qsc/DEMO/VGGFace_TF/data_preCategory/fold_4/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/home/qsc/DEMO/VGGFace_TF/data_preCategory/fold_4/valid',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 16423 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [6]:
model=load_model("models/vggface_classifier/fold2/20180604fold2_weights.20-1.43.hdf5")

In [6]:
for i, layer in enumerate(model.layers):
   print(i, layer.name,layer.trainable)

0 input_1 False
1 conv1_1 True
2 conv1_2 True
3 pool1 True
4 conv2_1 True
5 conv2_2 True
6 pool2 True
7 conv3_1 True
8 conv3_2 True
9 conv3_3 True
10 pool3 True
11 conv4_1 True
12 conv4_2 True
13 conv4_3 True
14 pool4 True
15 conv5_1 True
16 conv5_2 True
17 conv5_3 True
18 pool5 True
19 global_average_pooling2d_1 True
20 dense_1 True


In [6]:
for layer in model.layers[:19]:
    layer.trainable = False
#for layer in model.layers[19:]:
    #layer.trainable=True
#model.layers[17].trainable = True
#model.layers[20].trainable = True
#model.layers[22].trainable = True
for i, layer in enumerate(model.layers):
   print(i, layer.name,layer.trainable)


0 input_1 False
1 conv1_1 False
2 conv1_2 False
3 pool1 False
4 conv2_1 False
5 conv2_2 False
6 pool2 False
7 conv3_1 False
8 conv3_2 False
9 conv3_3 False
10 pool3 False
11 conv4_1 False
12 conv4_2 False
13 conv4_3 False
14 pool4 False
15 conv5_1 False
16 conv5_2 False
17 conv5_3 False
18 pool5 False
19 global_average_pooling2d_1 True
20 dense_1 True


In [7]:
#opt = optimizers.SGD(lr=1e-5,momentum=0.9)
opt=optimizers.Adam(lr=1e-3)
model.compile(optimizer=opt,loss=focal_loss,metrics=['accuracy'])

In [8]:
hist=model.fit_generator(
        train_generator,callbacks=[checkpointer],shuffle=True,
        steps_per_epoch=2000,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=50)

Epoch 1/20


InvalidArgumentError: Incompatible shapes: [32,5,5] vs. [32,5]
	 [[Node: loss/dense_1_loss/Mul_1 = Mul[T=DT_FLOAT, _class=["loc:@training/Adam/gradients/loss/dense_1_loss/Mul_1_grad/Reshape_1"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/dense_1_loss/one_hot, loss/dense_1_loss/Pow)]]
	 [[Node: loss/mul/_387 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1688_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'loss/dense_1_loss/Mul_1', defined at:
  File "/home/qsc/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/qsc/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/qsc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.

In [11]:
with open('log_freeze15-18.txt','w') as f:
    f.write(str(hist.history))